In [1]:
import matplotlib.pyplot as plt

In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#from model import 
from flask import Flask, request, render_template, url_for
from pymongo import MongoClient
from bson import json_util

import requests

In [2]:
client = MongoClient()
db = client.yelp
bdb = db.businesses
rdb = db.reviews

In [3]:
state_summary_data = client.yelp.businesses.aggregate([{'$group':{'_id':'$state', 'rating': {'$avg': '$rating'}, 'count': {'$sum':1} } }, {'$sort': { 'count': -1} }, {'$match': {'count': {'$gte': 100} } } ] )

In [ ]:
city_summary_data = bdb.aggregate([{'$group':{'_id':'$state', 'rating': {'$avg': '$rating'}, 'count': {'$sum':1} } }, {'$sort': { 'count': -1} }, {'$match': {'count': {'$gte': 100} } } ] )
    

In [40]:
city_list = bdb.aggregate([{'$group': {'_id': {'city': '$city', 'state': '$state'}, 'count': {'$sum':1} } }])


In [23]:
for city in city_list:
    city.keys()

In [24]:
city

{'_id': {'city': 'Bellrose', 'state': 'NY'}, 'count': 1}

In [26]:
city['_id']['city']

'Bellrose'

In [27]:
city['_id']['state']

'NY'

In [32]:
city['count']

1

In [41]:
city_count = [[city['_id']['city'], city['_id']['state'], city['count']] for city in city_list ]

In [42]:
len(city_count)

1702

In [45]:
city_count = np.array(city_count)

In [47]:
city_count[city_count[:,1]=='MD']

array([['Park Hall', 'MD', '2'],
       ['Tilghman Island', 'MD', '9'],
       ['Tilghman', 'MD', '12'],
       ['Great Mills', 'MD', '10'],
       ['Callaway', 'MD', '11'],
       ['Knoxville', 'MD', '8'],
       ['Thurmont', 'MD', '8'],
       ['New Windsor', 'MD', '14'],
       ['Upperco', 'MD', '4'],
       ['Bel Aire', 'MD', '6'],
       ['Bel-Air', 'MD', '6'],
       ['Stevensville', 'MD', '39'],
       ['Bushwood', 'MD', '6'],
       ['Sparks', 'MD', '15'],
       ['Bel Air', 'MD', '523'],
       ['Joppa', 'MD', '69'],
       ['Patuxent River', 'MD', '18'],
       ['Lexington Park', 'MD', '301'],
       ['California', 'MD', '278'],
       ['Myersville', 'MD', '30'],
       ['Baldwin', 'MD', '15'],
       ['Bowleys Quarters', 'MD', '15'],
       ['Clements', 'MD', '9'],
       ['Union Bridge', 'MD', '14'],
       ['Kingsville', 'MD', '39'],
       ['essex', 'MD', '6'],
       ['Point of Rocks', 'MD', '18'],
       ['Westminster', 'MD', '216'],
       ['Reisterstown MD', 'MD', '6'

In [54]:
zip(np.arange(0,len(city_count),1), city_count[:,1])

In [53]:
np.arange(0,len(city_count),1)

array([   0,    1,    2, ..., 1699, 1700, 1701])

In [64]:
np.vstack([np.arange(0,len(city_count),1), city_count[:,0],]).T

ValueError: all the input array dimensions except for the concatenation axis must match exactly